In [13]:
import json
import time
import random
import uuid
import string
import psycopg2
from faker import Faker
import os

In [14]:
import sys
sys.path.append('/home/xeon/app/consts.py')
from consts import CONSTS

In [15]:
def read_distinct_restaurant_ids(host, port,database, user, password, table):
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host=host,
        port = port,
        database=database,
        user=user,
        password=password
    )
    
    try:
        # Create a cursor object to interact with the database
        cur = conn.cursor()

        # Execute the SQL query to retrieve distinct restaurant_id values
        cur.execute(f"SELECT DISTINCT restaurant_id FROM {table}")

        # Fetch all the distinct restaurant_id values
        distinct_restaurant_ids = cur.fetchall()

        # Commit the transaction
        conn.commit()

        # Return the distinct restaurant_ids as a list
        return [record[0] for record in distinct_restaurant_ids]

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error retrieving distinct restaurant_ids:", error)

    finally:
        # Close the cursor and connection
        cur.close()
        conn.close()


In [16]:
def generate_random_timestamp(after=None):
    if after:
        # Generate a random timestamp after the provided timestamp
        return random.randint(after, after + 100000)
      
    # Generate a random timestamp after 30.07.2023
    return random.randint(1677736800, 2688036800)

def save_counter(counter,file_path):
    # Save the counter to a file
    with open(file_path, "w") as file:
        file.write(str(counter))

In [17]:
def read_counter(file_path):
    try:
        with open(file_path, "r") as file:
            counter = int(file.read())
            return counter
    except FileNotFoundError:        
        return 1
    except ValueError:
        print("Invalid counter value in the file.")
        return -1

In [19]:

restaurant_ids = read_distinct_restaurant_ids(
        CONSTS['POSTGRE_METADATA_SERVER'], \
        CONSTS['POSTGRE_METADATA_PORT'], \
        CONSTS['POSTGRE_METADATA_DB'], \
        CONSTS['POSTGRE_METADATA_USERNAME'], \
        CONSTS['POSTGRE_METADATA_PASSWORD'], \
        CONSTS['POSTGRE_METADATA_TB'])

In [91]:
# Incremental counter for message ID
def generate_message():
    counter = read_counter(CONSTS['COUNTER_FILE_NAME'])    

    fake = Faker()

    # Randomly select values
    restaurant_id = random.choice(restaurant_ids)
    adv_campaign_id = str(uuid.uuid4())
    adv_campaign_content = random.choice(CONSTS['RESTAURANT_CAMPAIGN'])
    adv_campaign_owner = fake.name()
    adv_campaign_owner_contact = fake.email()
    adv_campaign_datetime_start = generate_random_timestamp()
    adv_campaign_datetime_end = generate_random_timestamp(after=adv_campaign_datetime_start)
    datetime_created = int(time.time())

    # Create the message dictionary
    message = {        
        "restaurant_id": restaurant_id,
        "adv_campaign_id": adv_campaign_id,
        "adv_campaign_content": adv_campaign_content,
        "adv_campaign_owner": adv_campaign_owner,
        "adv_campaign_owner_contact": adv_campaign_owner_contact,
        "adv_campaign_datetime_start": adv_campaign_datetime_start,
        "adv_campaign_datetime_end": adv_campaign_datetime_end,
        "datetime_created": datetime_created
    }      

    # Increment and save the counter to file
    counter += 1    
    save_counter(counter,CONSTS['COUNTER_FILE_NAME'])    
    
    return f"id:{counter}, {json.dumps(message)}"

In [40]:
message = []

In [81]:
def save_to_file(path,how,messages):
    current_dir = os.getcwd()
    file_name = 'data'
    
    try:
        with open(path, how) as file:
            for m in messages:
                file.write(f"{m}\n")            
    except Exception as e:
        print(f"Something is wrong {e}")

In [44]:
def save_data(messages):
    current_dir = os.getcwd()
    file_path = f'{current_dir}/data'
    save_to_file(file_path,'w',messages)

In [53]:
def save_all_data(messages):
    current_dir = os.getcwd()
    file_path = f"{current_dir}/{CONSTS['ALL_MESSAGES']}"
    save_to_file(file_path,'a',messages)

In [64]:
import subprocess

def execute_bash_command(command):
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        output = result.stdout.strip()
        return result.returncode, output
    except subprocess.CalledProcessError as e:
        return e.returncode, e.stderr.strip()

def send_to_kafka():
    attemp = 1
    returncode = -1
    output = ""
    command = CONSTS['BASH_KAFKA']
    while returncode != 0:
        returncode, output = execute_bash_command(command)
        update_log(attemp,returncode,output)
        time.sleep(3)
        attemp+=1


In [65]:
def update_log(attemp, returncode, output):
    current_dir = os.getcwd()
    file_path = f'{current_dir}/{"log.txt"}'
    try:
        with open(file_path,'a') as file:
            line = f"Attemp: {attemp}, Return Code: {returncode}, Output: {output}"
            file.write(f"{line}\n")
            print(line)
    except Exception:    
        print('Something is wrong ')

In [89]:
def generator(pack_size,pack_number):
    for i in range(0,pack_number,1):
        messages = []
        for i in range(0,pack_size,1):
            messages.append(generate_message())        
        save_data(messages)
        save_all_data(messages)
        send_to_kafka()
        time.sleep(3)


In [92]:
generator(5,1)